In [38]:
from web3 import Web3, EthereumTesterProvider
from web3.auto import w3
from eth_account import Account

In [68]:
#w3 = Web3(EthereumTesterProvider())
#w3.is_connected()

w3 = Web3(Web3.HTTPProvider("https://sepolia.infura.io/v3/cbdf9e329a704c7d937be773ff2c3c2e"))
w3.is_connected()

True

In [69]:
def generate_new_wallet():
    # Generate a new account
    account = Account.create()
    # Get private key and address
    private_key = account.key.hex()
    address = account.address
    return private_key, address

In [78]:
private_key1, address1 = generate_new_wallet()
print(f"New Wallet - Address: {address1}, Private Key: {private_key1}")

private_key2, address2 = generate_new_wallet()
print(f"New Wallet - Address: {address2}, Private Key: {private_key2}")

New Wallet - Address: 0xC26CCf7c239d4F75eCD56692bA175a254b439683, Private Key: 0x9a01e950ea6ed278987e85aff4c2be198d8ed9fee2fb9a82229ca49a9ebf5bd8
New Wallet - Address: 0x920e98b91954282A363C08B5087B7c74B7aeA6e9, Private Key: 0xb8dca5cc1859a9a1d47d376bf2524a2631d9126050f5330d351b2f7098f2231a


In [86]:
def import_wallet(private_key):
    # Import an account using its private key
    wallet = Account.from_key(private_key)
    return wallet

# Example usage
existing_private_key = "0x2a96ea2cf3aa1bae024f8bb3610e15539f4e62b1647f3e181ba57eeb1f923b70"  # Replace with the actual private key
wallet1 = import_wallet(existing_private_key)
print(f"Imported Wallet - Address: {wallet1.address}")

existing_private_key = "0xecb8bd8142529761b34ad19adac6abf62b9988720280fcae5e0e12995efc17b0"  # Replace with the actual private key
wallet2 = import_wallet(existing_private_key)
print(f"Imported Wallet - Address: {wallet2.address}")

existing_private_key = "0x9a01e950ea6ed278987e85aff4c2be198d8ed9fee2fb9a82229ca49a9ebf5bd8"  # Replace with the actual private key
wallet3 = import_wallet(existing_private_key)
print(f"Imported Wallet - Address: {wallet3.address}")


Imported Wallet - Address: 0x633f4bA0e352f466591edDac63b8Cc232A7be544
Imported Wallet - Address: 0xfe86D01A532ee34f0ED04403D15Af2bA88a0d1c0
Imported Wallet - Address: 0xC26CCf7c239d4F75eCD56692bA175a254b439683


In [87]:
def get_balance(address):
    # Check if connected to Web3
    if w3.is_connected():
        # Convert address to checksum format
        checksum_address = w3.to_checksum_address(address)
        # Get balance in Wei
        balance_wei = w3.eth.get_balance(checksum_address)
        # Convert Wei to Ether
        return w3.from_wei(balance_wei, 'ether')
    else:
        return "Not connected to Web3"

# Example usage
balance = get_balance(wallet1.address)
print(f"Balance for {wallet1.address}: {balance} ETH")

balance = get_balance(wallet2.address)
print(f"Balance for {wallet2.address}: {balance} ETH")

balance = get_balance(wallet3.address)
print(f"Balance for {wallet3.address}: {balance} ETH")



Balance for 0x633f4bA0e352f466591edDac63b8Cc232A7be544: 0.489993593899201 ETH
Balance for 0xfe86D01A532ee34f0ED04403D15Af2bA88a0d1c0: 0.01 ETH
Balance for 0xC26CCf7c239d4F75eCD56692bA175a254b439683: 0 ETH


## Transactions

In [74]:
# Example to simulate internal transactions
def internal_transaction(wallets, from_address, to_address, amount):
    if wallets[from_address] >= amount:
        wallets[from_address] -= amount
        wallets[to_address] += amount
        return True
    return False

# Example wallet balances
wallets = {
    "0x633f4bA0e352f466591edDac63b8Cc232A7be544": 10,  # Replace with actual addresses and balances
    "0xfe86D01A532ee34f0ED04403D15Af2bA88a0d1c0": 5
}

# Simulate transaction
transaction_success = internal_transaction(wallets, "0x633f4bA0e352f466591edDac63b8Cc232A7be544", "0xfe86D01A532ee34f0ED04403D15Af2bA88a0d1c0", 1)
if transaction_success:
    print("Transaction successful.")
else:
    print("Insufficient funds.")


Transaction successful.


In [80]:
def send_transaction(from_wallet, to_address, amount):
    nonce = w3.eth.get_transaction_count(from_wallet.address)
    tx = {
        'nonce': nonce,
        'to': to_address,
        'value': w3.to_wei(amount, 'ether'),
        'gas': 21000,
        'gasPrice': w3.eth.gas_price
    }
    signed_tx = w3.eth.account.sign_transaction(tx, from_wallet.key)
    return w3.eth.send_raw_transaction(signed_tx.rawTransaction)

# Example Usage
tx_hash = send_transaction(wallet1, "0xfe86D01A532ee34f0ED04403D15Af2bA88a0d1c0", 0.01)
print(f"Transaction Hash: {w3.to_hex(tx_hash)}")

Transaction Hash: 0xc6331b89c2277dd7bc57095dd0ebf2c3d0a5ed85afd66d2634748e4b7813a88a


In [88]:
balance = get_balance(wallet1.address)
print(f"Balance for {wallet1.address}: {balance} ETH")

balance = get_balance(wallet2.address)
print(f"Balance for {wallet2.address}: {balance} ETH")

balance = get_balance(wallet3.address)
print(f"Balance for {wallet3.address}: {balance} ETH")

Balance for 0x633f4bA0e352f466591edDac63b8Cc232A7be544: 0.489993593899201 ETH
Balance for 0xfe86D01A532ee34f0ED04403D15Af2bA88a0d1c0: 0.01 ETH
Balance for 0xfe86D01A532ee34f0ED04403D15Af2bA88a0d1c0: 0.01 ETH


In [83]:
# https://sepolia.etherscan.io/tx/0x97348b6ff9c7f54314ee22eca095cb55a766f7fe1b66c70cbb8cabdc701a49a0#statechange
# Faucet: https://www.infura.io/faucet/sepolia

## Smart Contract

In [89]:
assert w3.is_connected(), "Failed to connect to Ethereum node."

# Contract ABI and Bytecode
contract_abi = """[
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "_giftValue",
				"type": "uint256"
			},
			{
				"internalType": "address payable[]",
				"name": "_contributors",
				"type": "address[]"
			}
		],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"inputs": [],
		"name": "contribute",
		"outputs": [],
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "contributors",
		"outputs": [
			{
				"internalType": "address payable",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "giftPurchased",
		"outputs": [
			{
				"internalType": "bool",
				"name": "",
				"type": "bool"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "giftValue",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address payable",
				"name": "vendor",
				"type": "address"
			}
		],
		"name": "purchaseGift",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "totalContributed",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]"""
contract_bytecode = """608060405234801562000010575f80fd5b5060405162000bf638038062000bf6833981810160405281019062000036919062000335565b81600181905550805f90805190602001906200005492919062000076565b505f60035f6101000a81548160ff021916908315150217905550505062000399565b828054828255905f5260205f20908101928215620000ef579160200282015b82811115620000ee578251825f6101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055509160200191906001019062000095565b5b509050620000fe919062000102565b5090565b5b808211156200011b575f815f90555060010162000103565b5090565b5f604051905090565b5f80fd5b5f80fd5b5f819050919050565b620001448162000130565b81146200014f575f80fd5b50565b5f81519050620001628162000139565b92915050565b5f80fd5b5f601f19601f8301169050919050565b7f4e487b71000000000000000000000000000000000000000000000000000000005f52604160045260245ffd5b620001b4826200016c565b810181811067ffffffffffffffff82111715620001d657620001d56200017c565b5b80604052505050565b5f620001ea6200011f565b9050620001f88282620001a9565b919050565b5f67ffffffffffffffff8211156200021a57620002196200017c565b5b602082029050602081019050919050565b5f80fd5b5f73ffffffffffffffffffffffffffffffffffffffff82169050919050565b5f6200025a826200022f565b9050919050565b6200026c816200024e565b811462000277575f80fd5b50565b5f815190506200028a8162000261565b92915050565b5f620002a6620002a084620001fd565b620001df565b90508083825260208201905060208402830185811115620002cc57620002cb6200022b565b5b835b81811015620002f95780620002e488826200027a565b845260208401935050602081019050620002ce565b5050509392505050565b5f82601f8301126200031a576200031962000168565b5b81516200032c84826020860162000290565b91505092915050565b5f80604083850312156200034e576200034d62000128565b5b5f6200035d8582860162000152565b925050602083015167ffffffffffffffff8111156200038157620003806200012c565b5b6200038f8582860162000303565b9150509250929050565b61084f80620003a75f395ff3fe608060405260043610610054575f3560e01c8063023f4147146100585780630b4b1075146100825780633c83a282146100ac5780633cb5d100146100d6578063b663425314610112578063d7bb99ba1461013a575b5f80fd5b348015610063575f80fd5b5061006c610144565b604051610079919061043e565b60405180910390f35b34801561008d575f80fd5b5061009661014a565b6040516100a3919061043e565b60405180910390f35b3480156100b7575f80fd5b506100c0610150565b6040516100cd9190610471565b60405180910390f35b3480156100e1575f80fd5b506100fc60048036038101906100f791906104b8565b610162565b6040516101099190610522565b60405180910390f35b34801561011d575f80fd5b5061013860048036038101906101339190610565565b61019c565b005b6101426102db565b005b60025481565b60015481565b60035f9054906101000a900460ff1681565b5f8181548110610170575f80fd5b905f5260205f20015f915054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b60015460025410156101e3576040517f08c379a00000000000000000000000000000000000000000000000000000000081526004016101da906105ea565b60405180910390fd5b60035f9054906101000a900460ff1615610232576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161022990610652565b60405180910390fd5b61023b33610386565b61027a576040517f08c379a0000000000000000000000000000000000000000000000000000000008152600401610271906106e0565b60405180910390fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc4790811502906040515f60405180830381858888f193505050501580156102bd573d5f803e3d5ffd5b50600160035f6101000a81548160ff02191690831515021790555050565b60035f9054906101000a900460ff161561032a576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161032190610652565b60405180910390fd5b5f341161036c576040517f08c379a00000000000000000000000000000000000000000000000000000000081526004016103639061076e565b60405180910390fd5b3460025f82825461037d91906107b9565b92505081905550565b5f805f90505b5f8054905081101561041c578273ffffffffffffffffffffffffffffffffffffffff165f82815481106103c2576103c16107ec565b5b905f5260205f20015f9054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff160361040f576001915050610421565b808060010191505061038c565b505f90505b919050565b5f819050919050565b61043881610426565b82525050565b5f6020820190506104515f83018461042f565b92915050565b5f8115159050919050565b61046b81610457565b82525050565b5f6020820190506104845f830184610462565b92915050565b5f80fd5b61049781610426565b81146104a1575f80fd5b50565b5f813590506104b28161048e565b92915050565b5f602082840312156104cd576104cc61048a565b5b5f6104da848285016104a4565b91505092915050565b5f73ffffffffffffffffffffffffffffffffffffffff82169050919050565b5f61050c826104e3565b9050919050565b61051c81610502565b82525050565b5f6020820190506105355f830184610513565b92915050565b61054481610502565b811461054e575f80fd5b50565b5f8135905061055f8161053b565b92915050565b5f6020828403121561057a5761057961048a565b5b5f61058784828501610551565b91505092915050565b5f82825260208201905092915050565b7f496e73756666696369656e742066756e647320636f6c6c65637465642e0000005f82015250565b5f6105d4601d83610590565b91506105df826105a0565b602082019050919050565b5f6020820190508181035f830152610601816105c8565b9050919050565b7f4769667420616c7265616479207075726368617365642e0000000000000000005f82015250565b5f61063c601783610590565b915061064782610608565b602082019050919050565b5f6020820190508181035f83015261066981610630565b9050919050565b7f4f6e6c7920636f6e7472696275746f72732063616e20707572636861736520745f8201527f686520676966742e000000000000000000000000000000000000000000000000602082015250565b5f6106ca602883610590565b91506106d582610670565b604082019050919050565b5f6020820190508181035f8301526106f7816106be565b9050919050565b7f436f6e747269627574696f6e206d757374206265206d6f7265207468616e20305f8201527f2e00000000000000000000000000000000000000000000000000000000000000602082015250565b5f610758602183610590565b9150610763826106fe565b604082019050919050565b5f6020820190508181035f8301526107858161074c565b9050919050565b7f4e487b71000000000000000000000000000000000000000000000000000000005f52601160045260245ffd5b5f6107c382610426565b91506107ce83610426565b92508282019050808211156107e6576107e561078c565b5b92915050565b7f4e487b71000000000000000000000000000000000000000000000000000000005f52603260045260245ffdfea26469706673582212202969db45bf64160edb7e42f61a4afc2f33eff9261e93744684091e7dfbe2eebc64736f6c63430008160033"""

# Create Contract instance
Contract = w3.eth.contract(abi=contract_abi, bytecode=contract_bytecode)

# Account details for deployment
account_address = '0x633f4bA0e352f466591edDac63b8Cc232A7be544'
private_key = '0x2a96ea2cf3aa1bae024f8bb3610e15539f4e62b1647f3e181ba57eeb1f923b70'

# Build Transaction (100000000000000 wei = 0.001 ETH)
construct_txn = Contract.constructor(1000000000000000, ["0x633f4bA0e352f466591edDac63b8Cc232A7be544", "0xfe86D01A532ee34f0ED04403D15Af2bA88a0d1c0", "0xfe86D01A532ee34f0ED04403D15Af2bA88a0d1c0"]).build_transaction({
    'from': account_address,
    'nonce': w3.eth.get_transaction_count(account_address),
    'gas': 2000000,
    'gasPrice': w3.to_wei('50', 'gwei')
})

# Sign Transaction
signed_txn = w3.eth.account.sign_transaction(construct_txn, private_key)

# Send Transaction
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)

# Wait for Transaction Receipt
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Contract Deployed at Address: {tx_receipt.contractAddress}")


Contract Deployed at Address: 0xA333b344c513E6f3Cf6BBb9611C5F7506AE14476


### Use smart contract by contributing to gift

In [92]:
# Contract ABI and Address
contract_abi = """[
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "_giftValue",
				"type": "uint256"
			},
			{
				"internalType": "address payable[]",
				"name": "_contributors",
				"type": "address[]"
			}
		],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"inputs": [],
		"name": "contribute",
		"outputs": [],
		"stateMutability": "payable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "contributors",
		"outputs": [
			{
				"internalType": "address payable",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "giftPurchased",
		"outputs": [
			{
				"internalType": "bool",
				"name": "",
				"type": "bool"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "giftValue",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address payable",
				"name": "vendor",
				"type": "address"
			}
		],
		"name": "purchaseGift",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "totalContributed",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]"""
contract_address = Web3.to_checksum_address('0xA333b344c513E6f3Cf6BBb9611C5F7506AE14476')  # Replace with your contract's address

# Create the contract instance
contract = w3.eth.contract(address=contract_address, abi=contract_abi)

# Sender's account details
sender_address = '0x633f4bA0e352f466591edDac63b8Cc232A7be544'
private_key = '0x2a96ea2cf3aa1bae024f8bb3610e15539f4e62b1647f3e181ba57eeb1f923b70'

# Amount to contribute (in Wei)
amount_to_contribute = w3.to_wei(0.0005, 'ether')  # Example: Sending 1 ETH

# Create the transaction
contribute_txn = contract.functions.contribute().build_transaction({
    'from': sender_address,
    'value': amount_to_contribute,
    'nonce': w3.eth.get_transaction_count(sender_address),
    'gas': 200000,
    'gasPrice': w3.to_wei('50', 'gwei')
})

# Sign the transaction
signed_txn = w3.eth.account.sign_transaction(contribute_txn, private_key)

# Send the transaction
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)

# Wait for the transaction to be mined
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Transaction successful with hash: {tx_receipt.transactionHash.hex()}")


Transaction successful with hash: 0x31b70fe815a44b9a358c5d29a402f8f680603e8993a39beb5a99d40481ff800e


In [94]:
balance = get_balance(wallet1.address)
print(f"Balance for {wallet1.address}: {balance} ETH")

balance = get_balance(wallet2.address)
print(f"Balance for {wallet2.address}: {balance} ETH")

balance = get_balance(wallet3.address)
print(f"Balance for {wallet3.address}: {balance} ETH")

Balance for 0x633f4bA0e352f466591edDac63b8Cc232A7be544: 0.455200843899201 ETH
Balance for 0xfe86D01A532ee34f0ED04403D15Af2bA88a0d1c0: 0.01 ETH
Balance for 0xC26CCf7c239d4F75eCD56692bA175a254b439683: 0 ETH
